# Demo - Deployment Test

The following notebook allows us to manually test a model deployment endpoint with a test image and visulaise the result

## Set Model Endpoint 
Choose which endpoint that will be tested

### Define Endpoint
Set the AML endpoint to test

In [ ]:
# endpoint url is found in the AML studio. You must append the probaility argument in the form url/score?prob=0.5
endpoint = ''

enable_auth = True
auth = ''

# Test image path
test_img = f'./test_image.jpg'

## Load Image

In [ ]:
img = open(test_img, 'rb').read()

## Score Image

In [ ]:
# Call the endpoint
import requests
import json

headers = { 'Content-Type':'application/json' }

if enable_auth:
    headers['Authorization']=f'Bearer {auth}'

resp = requests.post(endpoint, img, headers=headers)
print(resp)
results = resp.text
print(results)

## Present Results

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from PIL import Image
import numpy as np

In [ ]:
img_np=mpimg.imread(test_img)
img = Image.fromarray(img_np.astype('uint8'),'RGB')
x, y = img.size
IMAGE_SIZE = (x/50,y/50)
plt.figure(figsize=IMAGE_SIZE)

### Plot Result

In [ ]:
fig,ax = plt.subplots(1)
# Display the image
ax.imshow(img_np)

# draw box and label for each detection 
detections = json.loads(results)
for detect in detections:
    label = detect['label']
    box = detect['bounding_box']
    ymin, xmin, ymax, xmax = box[0], box[1], box[2], box[3]
    topleft_x, topleft_y = x * xmin, y * ymin
    width, height = x * (xmax - xmin), y * (ymax - ymin)
    print('{}: {}, {}, {}, {}'.format(detect['label'], topleft_x, topleft_y, width, height))

    color = 'white'
    rect = patches.Rectangle((topleft_x, topleft_y), width, height, 
                             linewidth=2, edgecolor=color,facecolor='none')

    ax.add_patch(rect)
    plt.text(topleft_x, topleft_y, label, color=color)

plt.show()